In [3]:
from numba import vectorize, cuda, njit, jit
import numpy as np
import math

In [13]:
N_particles = 800
N_run = 50
configuration = np.random.uniform(0, 4, (N_particles, 3))
sigma = np.array([1] * N_particles).astype(np.float64)
sigma_arr = 0.5 * (sigma[:, None] + sigma)
sigma_arr_6 = sigma_arr**6
sigma_arr_12 = sigma_arr**12
epsilon = np.array([1] * N_particles).astype(np.float64)
epsilon_arr = np.sqrt(epsilon[:, None] * epsilon)
cutoff_lj = 3.5
switch_width_lj = 1
switch_start_lj = 2.5

# Numpy as comparison  
I tried just doing calculations on upper triangualar matrix but was not faster.
It's kind of a mess because i first put force calc into the func aswell.

In [ ]:
def lj_numpy(configuration):
    #define potential
    def lj_pot(distances, mask1):
        output = (4 * epsilon_arr[mask1] * sigma_arr[mask1]**6 / distances**6
               * (sigma_arr[mask1]**6 / distances**6 - 1))
        return output

    # define switchfunction S1
    def switch_S1(distances):
        output = 2 * (distances - cutoff_lj) / switch_width_lj**3 + 3 * (distances - cutoff_lj) / switch_width_lj**2
        return output
    
#     # define part of the force depending only on distance
#     def lj_force_d_part(distances, mask):
#         output = (24 * epsilon_arr[mask] * sigma_arr[mask]**6 / distances**8 
#                   * (2 * sigma_arr[mask]**6 / distances**6 - 1))
#         return output

#     # define differential of switchfunction S1
#     def dswitch_S1__mult_pot(distances, mask):
#         output = (6 / (cutoff_lj * - switch_start_lj) / distances
#                   * (((distances - cutoff_lj) / switch_width_lj)**2 + (distances - cutoff_lj) / switch_width_lj)
#                   * lj_pot(distances, mask))
#         return output

    # calculate distance vectors and distances, init outputs
    potential = np.linalg.norm(configuration[:, None, :] - configuration[None, :, :], axis=-1)
#     distances = np.linalg.norm(distance_vectors, axis=-1)


    # get mask for switching function and triangular matrices exluding diagonal
    mask0 = (potential > 0) & (potential < switch_start_lj)
    mask1 = (potential > switch_start_lj) & (potential < cutoff_lj)

    # caculate potential
    potential[mask0] = lj_pot(potential[mask0], mask0)
    potential[mask1] = potential[mask1] * switch_S1(potential[mask1])
    potential = np.sum(potential, axis=-1)
    
#     # calculate forces
#     forces = np.empty_like(distance_vectors)
#     forces[:, :, 0] = 0
#     forces[:, :, 0][mask0] = lj_force_d_part(distances[mask0], mask0)
#     forces[:, :, 0][mask1] *= switch_S1(distances[mask1])
#     forces[:, :, 0][mask1] += dswitch_S1__mult_pot(distances[mask1], mask1)
#     for i in range(1, 3):
#         forces[:, :, i] = forces[:, :, 0]
#     forces *= distance_vectors
#     forces = np.sum(forces, axis=-1)
    return potential

In [ ]:
%%timeit
for _ in range(N_run):
    a = lj_numpy(configuration)

# Running over every particle twice

In [28]:
@njit(parallel=True)
def lj_potential_pairwise(distance, sigma_lj, epsilon_lj):
    # calculate potential between 0 and switch region
    if(distance <= cutoff_lj - switch_width_lj) and (distance > 0):
        phi = 4. * epsilon_lj * sigma_lj**6 / distance**6 * (sigma_lj**6 / distance**6 - 1)
        return phi
    
    # calculate potential in switch region
    elif (distance > cutoff_lj - switch_width_lj) and distance <= cutoff_lj:
        phi =  (4. * epsilon_lj * sigma_lj**6 / distance**6 * (sigma_lj**6 / distance**6 - 1)
                * (2 * ((distance - cutoff_lj) / switch_width_lj)**3 + 3 * ((distance - cutoff_lj) / switch_width_lj)**2))
        return phi
    
    # set rest to 0
    else:
        return 0.
        
@njit
def lj_potential_numba1(x):
    potential_output = np.zeros(len(x))
    for i in range(len(x)):
        for j in range(len(x)):
            sigma_lj = sigma_arr[i, j]
            epsilon_lj = epsilon_arr[i, j]
            distance_vector = x[i, :] - x[j, :]
            distance = np.linalg.norm(distance_vector)
            potential_output[i] += lj_potential_pairwise(distance, sigma_lj, epsilon_lj)
    return potential_output

In [ ]:
%%timeit
for _ in range(N_run):
    a = lj_potential_numba1(configuration)

# Running over every particle once

In [ ]:
@njit
def lj_potential_numba(x, sigma_arr, epsilon_arr):
    potential_output = np.zeros(len(x))
    for i in range(len(x)):
        for j in range(i, len(x)):
            sigma_lj = sigma_arr[i, j]
            epsilon_lj = epsilon_arr[i, j]
            distance_vector = x[i, :] - x[j, :]
            distance = np.linalg.norm(distance_vector)
            pot = lj_potential_pairwise(distance, sigma_lj, epsilon_lj)
            potential_output[i] += pot
            potential_output[j] += pot
    return potential_output

In [ ]:
%%timeit
for _ in range(N_run):
    a = lj_potential_numba(configuration, sigma_arr, epsilon_arr)

# No Precomputing sigma and epsilon

In [ ]:
@njit
def lj_potential_numba2(x):
    potential_output = np.zeros(len(x))
    for i in range(len(x)):
        for j in range(i, len(x)):
            sigma_lj = 0.5 * (sigma[i] + sigma[j]) 
            epsilon_lj = math.sqrt(epsilon[i] * epsilon[j])
            distance_vector = x[i, :] - x[j, :]
            distance = np.linalg.norm(distance_vector)
            potential_output[i] += lj_potential_pairwise(distance, sigma_lj, epsilon_lj)
            potential_output[j] += lj_potential_pairwise(distance, sigma_lj, epsilon_lj)
    return potential_output

In [ ]:
%%timeit
for _ in range(N_run):
    a = lj_potential_numba2(configuration)

# Precomputing distances

In [ ]:
def distances(x):
    return x[:, None, :] - x[None, :, :]

In [ ]:
%timeit distances(configuration)

In [ ]:
@njit
def distances_numba(x):
    output = np.zeros((x.shape[0], x.shape[0]))
    for i in range(len(x)):
        for j in range(i, len(x)):
            output[i, j] = np.linalg.norm(x[i, :] - x[j, :])

In [ ]:
%timeit distances_numba(configuration)

# Computing potential with distances precomputed

In [31]:
@njit(parallel=True)
def lj_potential_numba3(distances):
    for i in range(len(distances)):
        for j in range(i + 1, len(distances) - 1):
            sigma_lj = 0.5 * (sigma[i] + sigma[j]) 
            epsilon_lj = math.sqrt(epsilon[i] * epsilon[j])
            pot = lj_potential_pairwise(distances[i, j], sigma_lj, epsilon_lj)
            distances[i, j] = pot
            distances[i, j] = pot
    return distances


In [32]:
%%timeit
for _ in range(N_run):
    distances = np.linalg.norm(configuration[:, None, :] - configuration[None, :, :], axis=-1)
    a = lj_potential_numba3(distances)

1.23 s ± 1.46 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Precomputed distances with numpy masking

In [ ]:
@vectorize(['float64(float64, float64, float64)'], target='parallel')
def pot_vectorize(distance, sigma_lj, epsilon_lj):
    phi = 4. * epsilon_lj * sigma_lj**6 / distance**6 * (sigma_lj**6 / distance**6 - 1)
    return phi


@vectorize(['float64(float64, float64, float64)'], target='parallel')
def pot_vectorize_switch(distance, sigma_lj, epsilon_lj):
    phi = (4. * epsilon_lj * sigma_lj**6 / distance**6 * (sigma_lj**6 / distance**6 - 1)
           * (2 * ((distance - cutoff_lj) / switch_width_lj)**3 + 3 * ((distance - cutoff_lj) / switch_width_lj)**2))
    return phi

@jit
def lj_potential_numba_masking(configuration):
    distances = np.linalg.norm(configuration[:, None, :] - configuration[None, :, :], axis=-1)
    output = np.zeros(distances.shape)
    mask_triu = np.triu_indices(len(configuration), 1)
    mask_tril = np.tril_indices(len(configuration), -1)
    output[mask_triu] = distances[mask_triu]
    mask0 = (output < switch_start_lj) & (output > 0)
    mask1 = (output > switch_start_lj) & (output < cutoff_lj) 
    output[mask0] = pot_vectorize(distances[mask0], sigma_arr[mask0], epsilon_arr[mask0])
    output[mask1] = pot_vectorize_switch(distances[mask1], sigma_arr[mask1], epsilon_arr[mask1])
    output[mask_tril] = output[mask_triu]
    output = np.sum(output, axis=-1)
    return output

In [ ]:
%%timeit
for _ in range(N_run):
    a = lj_potential_numba_masking(configuration)